# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into deep neural networks for NLP!

## Objectives

You will be able to:

- Train a Word2Vec model and transform words into vectors 
- Obtain most similar words by using methods associated with word vectors 


## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec model. Then, you'll move onto building neural networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, import the data. The data is stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a GitHub repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json()` function from Pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've imported the data, inspect the first few rows of the DataFrame to see what your data looks like. 

In [2]:
df = pd.read_json('News_Category_Dataset_v2.json', lines = True)
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data -- `headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `'combined_text'` that consists of the data from the `'headline'` column plus a space character (`' '`) plus the data from the `'short_description'` column 
* Use the `'combined_text'` column's `.map()` method and pass in `word_tokenize`. Store the result returned in `data` 

In [4]:
df['combined_text'] = df['headline'] + ' ' + df['short_description']
data = df['combined_text'].map(word_tokenize)
data.head()

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Inspect the first 5 items in `data` to see how everything looks. 

In [5]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from `gensim`. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1` 
    * The number of threads to use during training, `workers=4`

In [6]:
model = Word2Vec(data, size = 100, window = 5, min_count = 1, workers = 4)

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call the `.train()` method on your model and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count` 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [7]:
model.train(data, total_examples = model.corpus_count, epochs = 10)

(55562503, 67352790)

Great! You now have a fully trained model! The word vectors themselves are stored in the `Word2VecKeyedVectors` instance, which is stored in the `.wv` attribute. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [8]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this by passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [9]:
wv.most_similar('Texas')

[('Ohio', 0.8008663654327393),
 ('Oregon', 0.798102617263794),
 ('Massachusetts', 0.7843394875526428),
 ('California', 0.7760922908782959),
 ('Oklahoma', 0.770129919052124),
 ('Pennsylvania', 0.7682232856750488),
 ('Arkansas', 0.7663991451263428),
 ('Florida', 0.7628408074378967),
 ('Michigan', 0.7615156173706055),
 ('Illinois', 0.7565325498580933)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `.most_similar()` method's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [11]:
wv.most_similar(negative = 'Texas')

[('UIUC', 0.4412378668785095),
 ('Adulterers', 0.3882267475128174),
 ('maitre', 0.3868247866630554),
 ('Parent/Grandparent', 0.38674184679985046),
 ('unachievable', 0.38372802734375),
 ('Headstrong', 0.3781481385231018),
 ('Admon', 0.3700985908508301),
 ('aboutagirl1', 0.3624870181083679),
 ('Gladly', 0.36144500970840454),
 ('memory-loss', 0.35777798295021057)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [12]:
wv['Texas']

array([ 2.3402388e+00,  3.3669610e+00,  1.6799094e-01,  6.2506425e-01,
        1.6168623e+00, -1.2922364e+00, -7.9780096e-01,  1.8945816e-01,
       -2.7135879e-01,  2.5412115e-01, -2.3069992e+00,  6.7259258e-01,
       -3.6724207e-01,  1.3197204e+00, -1.3405055e+00,  1.4328101e+00,
       -1.1647606e-02, -4.7096640e-01,  2.3982242e-01, -1.2627169e+00,
        1.1380228e+00,  1.7016906e-01, -2.3806753e+00,  7.1533191e-01,
        5.0310005e-02,  2.4964394e-01, -1.1162529e+00, -1.1519184e+00,
        8.7502712e-01,  3.3545333e-01,  2.2431591e+00, -8.5326123e-01,
        2.3296349e-01,  1.6135627e+00, -1.0628482e+00, -7.4540275e-01,
        3.8995710e-01,  4.5677412e-01,  2.5608785e+00, -1.9026575e+00,
       -7.4813819e-01, -7.4522334e-01,  1.6022582e+00, -1.7406321e-01,
        5.0221276e+00, -7.3696148e-01, -7.1082550e-01,  1.1965591e+00,
        9.8466402e-01,  1.0508523e+00, -9.9142659e-01,  2.5065293e+00,
        1.5453541e+00,  2.3405275e+00,  2.1935019e+00,  1.0798481e+00,
      

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [13]:
wv.vectors

array([[ 1.5409520e+00,  4.4217095e-01,  1.6648978e-02, ...,
         2.4413276e+00,  1.2377257e+00,  3.6323133e-01],
       [-2.0559599e-01,  8.8768923e-01, -2.2473654e-01, ...,
         2.8565204e+00,  1.7056018e+00, -1.7821115e-01],
       [ 8.6217624e-01,  1.4413673e+00, -7.0864889e-03, ...,
         8.4851289e-01,  2.6111338e+00, -1.0589616e+00],
       ...,
       [-1.0756719e-01,  4.4954646e-02,  1.6792695e-01, ...,
        -4.6169579e-02, -5.8968414e-02,  4.8375446e-02],
       [-4.8420850e-02, -2.8447937e-02,  6.1284266e-02, ...,
        -1.6077396e-02, -1.4184833e-04,  1.1005501e-02],
       [-9.7664870e-02,  4.3762881e-02,  1.1504756e-01, ...,
        -4.2749316e-02, -2.9721232e-02,  4.7432907e-02]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `.most_similar()` method and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [15]:
wv.most_similar(positive = ['king', 'woman'], negative = ['man'])

[('queen', 0.6041513681411743),
 ('jewel', 0.5994794368743896),
 ('princess', 0.5965747237205505),
 ('brunette', 0.5956114530563354),
 ('title', 0.5826456546783447),
 ('symbol', 0.5769343972206116),
 ('crown', 0.571063220500946),
 ('invention', 0.5622594952583313),
 ('goddess', 0.5563668012619019),
 ('Queen', 0.5516346096992493)]

As you can see from the output above, your model isn't perfect, but 'Queen' and 'Princess' are still in the top 5. As you can see from the other word in top 5, 'reminiscent' -- your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to create vectorized word embeddings!